In [1]:
import torchrl
from torchrl.utils import make_parallel_env

import torch
import numpy as np

from torchrl.a2c import RolloutStorage
from torchrl.a2c import ActorCritic
from collections import deque

import time

import pdb

In [2]:
torch.set_num_threads(1)
device = torch.device("cuda")

In [3]:
params = {}
params["rollout_len"] = 5
params["num_workers"] = 32

env = make_parallel_env("BreakoutNoFrameskip-v4", 
                        seed=1, num_workers=params["num_workers"], 
                        num_frame_stack=4, device=device, log_dir="results")

params["obs_shape"] = env.observation_space.shape
params["action_shape"] = env.action_space.n

params["total_steps"] = 1e7
params["discount_gamma"] = 0.99
params["gae_tau"] = 0.95
params["use_gae"] = False
params["entropy_coef"] = 0.01
params["value_coef"] = 0.5
params['num_in_channels'] = 4
params["max_grad_norm"] = 0.5
params["learning_rate"] = 7e-4
params["RMS_alpha"] = 0.99
params["RMS_eps"] = 1e-5
params["resume"] = False

Process Process-29:
Process Process-26:
Process Process-9:
Process Process-20:
Process Process-23:
Process Process-12:
Process Process-5:
Process Process-27:
Process Process-22:
Process Process-11:
Process Process-13:
Process Process-1:
Process Process-6:
Process Process-24:
Traceback (most recent call last):
Process Process-19:
Process Process-30:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/will/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Process Process-25:
Process Process-18:
  File "/home/will/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Process Process-21:
Traceback (most recent call last):
Process Process-3:
Process Process-10:
Process Process-7:
  File "/home/will/anaconda3/lib/python3.

  File "/home/will/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/will/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/will/Desktop/pytorchrl/torchrl/parallel/subproc_vec_env.py", line 14, in _worker
    cmd, data = remote.recv()
  File "/home/will/anaconda3/lib/python3.7/site-packages/gym/core.py", line 304, in step
    observation, reward, done, info = self.env.step(action)
  File "/home/will/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/will/Desktop/pytorchrl/torchrl/parallel/subproc_vec_env.py", line 14, in _worker
    cmd, data = remote.recv()
Traceback (most recent call last):
  File "/home/will/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/will/anaconda3/lib/python3.7/multiprocessing/connection.py", l

  File "/home/will/Desktop/pytorchrl/torchrl/parallel/subproc_vec_env.py", line 14, in _worker
    cmd, data = remote.recv()
KeyboardInterrupt
  File "/home/will/anaconda3/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/home/will/anaconda3/lib/python3.7/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/will/Desktop/pytorchrl/torchrl/parallel/subproc_vec_env.py", line 14, in _worker
    cmd, data = remote.recv()
  File "/home/will/anaconda3/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/home/will/Desktop/pytorchrl/torchrl/parallel/subproc_vec_env.py", line 14, in _worker
    cmd, data = remote.recv()
  File "/home/will/anaconda3/lib/python3.7/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
  File "/home/will/anaconda3/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chu

KeyboardInterrupt
  File "/home/will/Desktop/pytorchrl/torchrl/parallel/subproc_vec_env.py", line 14, in _worker
    cmd, data = remote.recv()
KeyboardInterrupt
KeyboardInterrupt
  File "/home/will/Desktop/pytorchrl/torchrl/parallel/subproc_vec_env.py", line 14, in _worker
    cmd, data = remote.recv()
  File "/home/will/anaconda3/lib/python3.7/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
  File "/home/will/anaconda3/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
KeyboardInterrupt
  File "/home/will/Desktop/pytorchrl/torchrl/utils/atari_wrappers.py", line 81, in step
    obs, reward, done, info = self.env.step(action)
  File "/home/will/anaconda3/lib/python3.7/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
  File "/home/will/Desktop/pytorchrl/torchrl/utils/atari_wrappers.py", line 136, in step
    obs, reward, done, info = self.env.step(action)
  Fil

In [4]:
obs = env.reset()

In [5]:
rollouts = RolloutStorage(params)
rollouts.obs[0].copy_(obs)
rollouts.to(device)

In [6]:
model = ActorCritic(params).to(device)

In [7]:
optimizer = torch.optim.RMSprop(model.parameters(),
                           params["learning_rate"],
                           alpha=params["RMS_alpha"],
                           eps=params["RMS_eps"]
                          )

In [8]:
num_steps = 0

In [10]:
episode_rewards = deque(maxlen=50)

t0 = time.time()

batch_size = int(params["num_workers"] * params["rollout_len"])
num_iter = int(params["total_steps"] // batch_size)
log_interval = 1000
log_iter = int(log_interval // batch_size)

rollouts.obs[0].copy_(obs)

for e in range(num_iter):
    #-------------------
    # Evaluation Network 
    #-------------------

    # collect rollout
    for step in range(params["rollout_len"]):
        # Sample actions
        with torch.no_grad():
            actions, values = model.act(rollouts.obs[step])

        # move a step
        states, rewards, dones, infos = env.step(actions)

        for info in infos:
            if 'episode' in info.keys():
                episode_rewards.append(info['episode']['r'])

        rollouts.insert(states, rewards, 1 - torch.from_numpy(dones.astype(np.float32)), actions, values.squeeze())

    # calculate the next value
    with torch.no_grad():
        next_value = model.get_value(rollouts.obs[-1]).squeeze()

    rollouts.compute_returns(next_value)
    # to cpu or gpu
    rollouts.to(device)
    

    #---------------
    # Update Network 
    #---------------
    
    batch_states = rollouts.obs[:-1].view(-1, *params["obs_shape"])
    batch_actions = rollouts.actions.view(-1)

    log_probs, values, entropy = model.eval_action(batch_states, batch_actions)
    
    advs = rollouts.returns[:-1].view(-1) - values.squeeze()
    value_loss = advs.pow(2).mean()
    policy_loss = -(advs.detach() * log_probs).mean()
    
    loss = policy_loss + params["value_coef"]*value_loss - params["entropy_coef"]*entropy


    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(
            model.parameters(), params["max_grad_norm"])
    optimizer.step()
    
    rollouts.after_update()

    # time for logging
    if e % log_iter == 0:
        if len(episode_rewards) > 0:
            print('Total steps: %d Returns %.2f/%.2f/%.2f/%.2f (mean/median/min/max)  %.2f steps/s' % (
                num_steps,
                np.mean(episode_rewards),
                np.median(episode_rewards),
                np.min(episode_rewards),
                np.max(episode_rewards),
                batch_size * log_iter / (time.time() - t0)
            ))
            t0 = time.time()
    
    num_steps += params["rollout_len"] * params["num_workers"]


Total steps: 3840 Returns 0.00/0.00/0.00/0.00 (mean/median/min/max)  357.91 steps/s
Total steps: 4800 Returns 0.17/0.00/0.00/1.00 (mean/median/min/max)  2410.84 steps/s
Total steps: 5760 Returns 0.43/0.00/0.00/2.00 (mean/median/min/max)  2386.24 steps/s
Total steps: 6720 Returns 0.79/0.00/0.00/3.00 (mean/median/min/max)  2175.06 steps/s
Total steps: 7680 Returns 0.81/0.00/0.00/3.00 (mean/median/min/max)  2185.31 steps/s
Total steps: 8640 Returns 0.83/0.00/0.00/3.00 (mean/median/min/max)  2254.92 steps/s
Total steps: 9600 Returns 0.85/0.00/0.00/3.00 (mean/median/min/max)  2228.51 steps/s
Total steps: 10560 Returns 0.88/0.50/0.00/3.00 (mean/median/min/max)  2377.41 steps/s
Total steps: 11520 Returns 1.12/1.00/0.00/3.00 (mean/median/min/max)  2312.85 steps/s
Total steps: 12480 Returns 1.30/1.00/0.00/4.00 (mean/median/min/max)  2320.14 steps/s
Total steps: 13440 Returns 1.42/1.00/0.00/4.00 (mean/median/min/max)  2517.69 steps/s
Total steps: 14400 Returns 1.38/1.00/0.00/4.00 (mean/median/mi

Total steps: 96000 Returns 1.32/1.00/0.00/10.00 (mean/median/min/max)  2079.48 steps/s
Total steps: 96960 Returns 1.28/1.00/0.00/10.00 (mean/median/min/max)  1998.86 steps/s
Total steps: 97920 Returns 1.08/0.00/0.00/10.00 (mean/median/min/max)  2058.15 steps/s
Total steps: 98880 Returns 0.98/0.00/0.00/4.00 (mean/median/min/max)  2053.30 steps/s
Total steps: 99840 Returns 1.00/0.00/0.00/4.00 (mean/median/min/max)  2201.03 steps/s
Total steps: 100800 Returns 1.00/0.00/0.00/4.00 (mean/median/min/max)  2085.92 steps/s
Total steps: 101760 Returns 1.08/0.50/0.00/4.00 (mean/median/min/max)  1975.02 steps/s
Total steps: 102720 Returns 0.94/0.00/0.00/4.00 (mean/median/min/max)  2061.03 steps/s
Total steps: 103680 Returns 1.04/0.00/0.00/5.00 (mean/median/min/max)  2125.40 steps/s
Total steps: 104640 Returns 1.08/0.00/0.00/5.00 (mean/median/min/max)  2045.46 steps/s
Total steps: 105600 Returns 1.06/0.00/0.00/6.00 (mean/median/min/max)  1896.31 steps/s
Total steps: 106560 Returns 1.38/0.00/0.00/10

Total steps: 187200 Returns 1.72/1.00/0.00/9.00 (mean/median/min/max)  2461.55 steps/s
Total steps: 188160 Returns 1.98/1.50/0.00/9.00 (mean/median/min/max)  2318.66 steps/s
Total steps: 189120 Returns 1.74/0.00/0.00/9.00 (mean/median/min/max)  2330.72 steps/s
Total steps: 190080 Returns 1.80/0.00/0.00/9.00 (mean/median/min/max)  2489.35 steps/s
Total steps: 191040 Returns 1.70/0.00/0.00/9.00 (mean/median/min/max)  2286.73 steps/s
Total steps: 192000 Returns 1.72/0.00/0.00/9.00 (mean/median/min/max)  2265.80 steps/s
Total steps: 192960 Returns 1.62/0.00/0.00/9.00 (mean/median/min/max)  2399.29 steps/s
Total steps: 193920 Returns 1.52/0.00/0.00/9.00 (mean/median/min/max)  2389.12 steps/s
Total steps: 194880 Returns 1.56/0.00/0.00/9.00 (mean/median/min/max)  2507.64 steps/s
Total steps: 195840 Returns 1.72/0.00/0.00/9.00 (mean/median/min/max)  2298.74 steps/s
Total steps: 196800 Returns 1.86/0.50/0.00/9.00 (mean/median/min/max)  2363.36 steps/s
Total steps: 197760 Returns 2.16/1.00/0.00/

Total steps: 277440 Returns 2.52/2.00/0.00/15.00 (mean/median/min/max)  2411.90 steps/s
Total steps: 278400 Returns 2.62/2.00/0.00/15.00 (mean/median/min/max)  2422.12 steps/s
Total steps: 279360 Returns 2.64/2.00/0.00/15.00 (mean/median/min/max)  2325.61 steps/s
Total steps: 280320 Returns 2.20/2.00/0.00/10.00 (mean/median/min/max)  2356.31 steps/s
Total steps: 281280 Returns 2.48/2.00/0.00/14.00 (mean/median/min/max)  2408.14 steps/s
Total steps: 282240 Returns 2.50/2.00/0.00/14.00 (mean/median/min/max)  2407.18 steps/s
Total steps: 283200 Returns 2.48/2.00/0.00/14.00 (mean/median/min/max)  2321.42 steps/s
Total steps: 284160 Returns 2.04/1.00/0.00/14.00 (mean/median/min/max)  2186.88 steps/s
Total steps: 285120 Returns 2.26/2.00/0.00/14.00 (mean/median/min/max)  2393.21 steps/s
Total steps: 286080 Returns 2.38/2.00/0.00/14.00 (mean/median/min/max)  2388.49 steps/s
Total steps: 287040 Returns 3.12/2.00/0.00/14.00 (mean/median/min/max)  2232.78 steps/s
Total steps: 288000 Returns 3.00

Total steps: 367680 Returns 2.34/1.00/0.00/13.00 (mean/median/min/max)  2501.12 steps/s
Total steps: 368640 Returns 2.28/1.00/0.00/10.00 (mean/median/min/max)  2291.73 steps/s
Total steps: 369600 Returns 2.12/1.00/0.00/10.00 (mean/median/min/max)  2410.52 steps/s
Total steps: 370560 Returns 2.24/1.00/0.00/10.00 (mean/median/min/max)  2473.09 steps/s
Total steps: 371520 Returns 2.68/1.00/0.00/13.00 (mean/median/min/max)  2391.66 steps/s
Total steps: 372480 Returns 2.54/1.00/0.00/13.00 (mean/median/min/max)  2307.77 steps/s
Total steps: 373440 Returns 2.48/1.00/0.00/13.00 (mean/median/min/max)  2431.97 steps/s
Total steps: 374400 Returns 2.44/1.00/0.00/13.00 (mean/median/min/max)  2504.21 steps/s
Total steps: 375360 Returns 2.60/1.00/0.00/13.00 (mean/median/min/max)  2340.52 steps/s
Total steps: 376320 Returns 2.60/1.00/0.00/13.00 (mean/median/min/max)  2539.99 steps/s
Total steps: 377280 Returns 2.92/1.50/0.00/13.00 (mean/median/min/max)  2359.87 steps/s
Total steps: 378240 Returns 2.70

...

Total steps: 2595840 Returns 275.26/290.50/73.00/424.00 (mean/median/min/max)  2605.09 steps/s
Total steps: 2596800 Returns 275.86/290.50/73.00/424.00 (mean/median/min/max)  2616.15 steps/s
Total steps: 2597760 Returns 275.86/290.50/73.00/424.00 (mean/median/min/max)  2604.54 steps/s
Total steps: 2598720 Returns 275.86/290.50/73.00/424.00 (mean/median/min/max)  2625.79 steps/s
Total steps: 2599680 Returns 274.16/279.50/73.00/424.00 (mean/median/min/max)  2589.26 steps/s
Total steps: 2600640 Returns 274.16/279.50/73.00/424.00 (mean/median/min/max)  2653.66 steps/s
Total steps: 2601600 Returns 273.84/279.50/73.00/424.00 (mean/median/min/max)  2527.64 steps/s
Total steps: 2602560 Returns 278.92/290.50/73.00/424.00 (mean/median/min/max)  2570.56 steps/s
Total steps: 2603520 Returns 280.94/296.50/73.00/424.00 (mean/median/min/max)  2553.89 steps/s
Total steps: 2604480 Returns 280.94/296.50/73.00/424.00 (mean/median/min/max)  2620.05 steps/s
Total steps: 2605440 Returns 280.94/296.50/73.00/4

Total steps: 2679360 Returns 312.00/335.00/48.00/406.00 (mean/median/min/max)  2292.88 steps/s
Total steps: 2680320 Returns 312.00/335.00/48.00/406.00 (mean/median/min/max)  2399.75 steps/s
Total steps: 2681280 Returns 312.00/335.00/48.00/406.00 (mean/median/min/max)  2373.49 steps/s
Total steps: 2682240 Returns 311.52/335.00/48.00/406.00 (mean/median/min/max)  2282.46 steps/s
Total steps: 2683200 Returns 311.52/335.00/48.00/406.00 (mean/median/min/max)  2405.87 steps/s
Total steps: 2684160 Returns 311.52/335.00/48.00/406.00 (mean/median/min/max)  2633.21 steps/s
Total steps: 2685120 Returns 311.56/335.00/48.00/406.00 (mean/median/min/max)  2530.63 steps/s
Total steps: 2686080 Returns 311.56/335.00/48.00/406.00 (mean/median/min/max)  2650.26 steps/s
Total steps: 2687040 Returns 309.32/335.00/43.00/406.00 (mean/median/min/max)  2594.40 steps/s
Total steps: 2688000 Returns 309.26/338.50/43.00/406.00 (mean/median/min/max)  2532.45 steps/s
Total steps: 2688960 Returns 312.58/341.00/43.00/4

Total steps: 2762880 Returns 317.68/355.50/43.00/417.00 (mean/median/min/max)  2695.26 steps/s
Total steps: 2763840 Returns 317.68/355.50/43.00/417.00 (mean/median/min/max)  2686.46 steps/s
Total steps: 2764800 Returns 317.68/355.50/43.00/417.00 (mean/median/min/max)  2665.55 steps/s
Total steps: 2765760 Returns 323.82/355.50/43.00/417.00 (mean/median/min/max)  2594.34 steps/s
Total steps: 2766720 Returns 323.82/355.50/43.00/417.00 (mean/median/min/max)  2634.57 steps/s
Total steps: 2767680 Returns 323.82/355.50/43.00/417.00 (mean/median/min/max)  2721.76 steps/s
Total steps: 2768640 Returns 323.82/355.50/43.00/417.00 (mean/median/min/max)  2637.64 steps/s
Total steps: 2769600 Returns 323.82/355.50/43.00/417.00 (mean/median/min/max)  2694.74 steps/s
Total steps: 2770560 Returns 323.82/355.50/43.00/417.00 (mean/median/min/max)  2692.13 steps/s
Total steps: 2771520 Returns 323.82/355.50/43.00/417.00 (mean/median/min/max)  2640.64 steps/s
Total steps: 2772480 Returns 321.02/352.00/43.00/4

Total steps: 2846400 Returns 310.88/352.50/45.00/425.00 (mean/median/min/max)  2840.86 steps/s
Total steps: 2847360 Returns 310.88/352.50/45.00/425.00 (mean/median/min/max)  2827.24 steps/s
Total steps: 2848320 Returns 310.88/352.50/45.00/425.00 (mean/median/min/max)  2832.61 steps/s
Total steps: 2849280 Returns 305.30/340.50/45.00/425.00 (mean/median/min/max)  1585.64 steps/s
Total steps: 2850240 Returns 305.30/340.50/45.00/425.00 (mean/median/min/max)  2400.50 steps/s
Total steps: 2851200 Returns 305.30/340.50/45.00/425.00 (mean/median/min/max)  2309.59 steps/s
Total steps: 2852160 Returns 305.30/340.50/45.00/425.00 (mean/median/min/max)  2015.31 steps/s
Total steps: 2853120 Returns 308.54/340.50/45.00/425.00 (mean/median/min/max)  2594.33 steps/s
Total steps: 2854080 Returns 308.54/340.50/45.00/425.00 (mean/median/min/max)  2733.24 steps/s
Total steps: 2855040 Returns 308.54/340.50/45.00/425.00 (mean/median/min/max)  2759.21 steps/s


KeyboardInterrupt: 